In [28]:
from pycorenlp import StanfordCoreNLP 
import TreeKernel as tk
import TreeBuild as tb
import numpy as np
import pandas as pd
import pickle

nlp = StanfordCoreNLP('http://localhost:9000')

In [29]:
def _getNLPToks_(rawSentence):
    try:
        output = nlp.annotate(rawSentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,depparse',
            'outputFormat': 'json'
        })
    except UnicodeDecodeError:
        sentence = unidecode(rawSentence)
        output = nlp.annotate(sentence, properties={
            'annotators': 'tokenize,ssplit,pos,parse,depparse',
            'outputFormat': 'json'
        })
    dependencies = output['sentences'][0]['basicDependencies']
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {'deps':dependencies, 'toks':tokens, 'parse':parse}


In [30]:
feature_vect = {}

with open('input/stanfordData_train1.nlp', 'rb') as handle:
    count = 0
    while True:
        if (count % 10000 == 0):
            print(count)

        try:
            unit = pickle.load(handle)
            # ST Syntax score
            (rscore, nscore) = tk._CollinsDuffy_(unit['q1']['parse'], unit['q2']['parse'], 1, 1, 0)
            feature_vect[unit['id']] = {'id':unit['id'], 'cdNorm':nscore, 'cdRaw':rscore}
            
        except EOFError:
            break
        except:
            print("Quote error on: %d", unit['id'])
            
            q1_stanford = _getNLPToks_(unit['q1']['raw'].replace('"','').replace("'",''))
            q2_stanford = _getNLPToks_(unit['q2']['raw'].replace('"','').replace("'",''))

            tree_1 = tb.tree()
            tree_2 = tb.tree()

            # Generate a tree structure
            tb._generateTree_(q1_stanford['parse'], tree_1)
            tb._generateTree_(q2_stanford['parse'], tree_2)

            # Flip the trees
            tb._flipTree_(tree_1)
            tb._flipTree_(tree_2)

            (rscore, nscore) = tk._CollinsDuffy_(tree_1,
                                                 tree_2,
                                                 1, 1, 0)
            
            feature_vect[unit['id']] = {'id':unit['id'], 'cdNorm':nscore, 'cdRaw':rscore}
            print("Quote error resolved")
            pass 

        count += 1

0
10000
20000
Quote error on: %d 23989
Quote error resolved
Quote error on: %d 29784
Quote error resolved
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
Quote error on: %d 247982
Quote error resolved
250000
Quote error on: %d 257478
Quote error resolved
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
Quote error on: %d 356092
Quote error resolved
360000
370000
380000
390000
400000


In [50]:
df_feature = pd.DataFrame.from_dict(feature_vect)
df_feature = df_feature.transpose()

In [57]:
df_feature[['id']] = df_feature[['id']].astype(int)

In [64]:
df_feature.to_csv('input/collins_duffy.csv', index=False)

In [63]:
df_train = pd.read_csv('input/train.csv')

TypeError: parser_f() got an unexpected keyword argument 'path_or_buf'

In [22]:
qsample = df_train.ix[23989]

print(qsample['question1'])
print(qsample['question1'].replace('"','').replace("'",''))

q1_stanford = _getNLPToks_(qsample['question1'].replace('"','').replace("'",''))
q2_stanford = _getNLPToks_(qsample['question2'])

tree_1 = tb.tree()
tree_2 = tb.tree()

# Generate a tree structure
tb._generateTree_(q1_stanford['parse'], tree_1)
tb._generateTree_(q2_stanford['parse'], tree_2)

# Flip the trees
tb._flipTree_(tree_1)
tb._flipTree_(tree_2)

(rscore, nscore) = tk._CollinsDuffy_(tree_1, tree_2, 1, 1, 0) 
print(rscore)
print(nscore)

for i in tree_1.items():
    print(i)
    

"Tomorrow is my exam, but I don't care because a single sheet of paper can't decide my future". Isn't this very easy to say?
Tomorrow is my exam, but I dont care because a single sheet of paper cant decide my future. Isnt this very easy to say?
7.0
0.264575131106
(0, {'indent': 0, 'childrenTok': 'S', 'parid': -1, 'posOrTok': 'ROOT', 'children': [1], 'curid': 0})
(1, {'indent': 2, 'childrenTok': ['S', 'CC', 'S'], 'parid': 0, 'posOrTok': 'S', 'children': [2, 9, 11], 'curid': 1})
(2, {'indent': 4, 'childrenTok': ['NP', 'VP'], 'parid': 1, 'posOrTok': 'S', 'children': [3, 6], 'curid': 2})
(3, {'indent': 6, 'childrenTok': ['NN'], 'parid': 2, 'posOrTok': 'NP', 'children': [4], 'curid': 3})
(4, {'indent': 8, 'childrenTok': ['Tomorrow'], 'parid': 3, 'posOrTok': 'NN', 'children': [5], 'curid': 4})
(5, {'indent': 8, 'childrenTok': [], 'parid': 4, 'posOrTok': 'Tomorrow', 'children': [], 'curid': 5})
(6, {'indent': 6, 'childrenTok': ['VBZ'], 'parid': 2, 'posOrTok': 'VP', 'children': [7], 'curid': 6

In [1]:
feature_vect

NameError: name 'feature_vect' is not defined